## PREGUNTAS
- ¿Existe una correlación significativa entre la tasa de suicidios y el promedio de
ingresos a nivel nacional en diferentes países?, si es que existe ¿Cómo ha
evolucionado a lo largo de la última década?
- ¿Cómo han evolucionado las tasas de suicidios en diferentes países a lo largo de la
última década? (Países con mayor aumento y países con menor aumento)
- ¿Cómo influye la desigualdad de ingresos en la tasa de suicidios dentro de un país
en comparación con el nivel global?
- ¿Existe una relación entre la tasa de suicidios de un país por cada 100.000
habitantes y el nivel educativo de estos?
- ¿Cómo afecta la religión a la tasa de suicidios en diferentes países?
- ¿Cómo influye el acceso a la tecnología en la tasa de suicidios en diferentes grupos
demográficos?
- ¿Se relaciona la tasa de suicidios con la esperanza de vida de los países?

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [90]:
df_suicidio_oms= pd.read_excel('Datos/data.xlsx',skiprows=2)
df_suicidio_oms.info

pd.set_option('display.max_columns', None)
df_suicidio_oms.dropna(thresh=0.2) #Eliminar NaN
df_suicidio_oms.head(10).columns[0]


'IndicatorCode'

In [86]:
columnas= df_suicidio_oms.columns
columnas_constantes=[]
for columna in columnas:
    c=0
    for fila in df_suicidio_oms[columna]:
        if df_suicidio_oms[columna][0] == fila:
            c+=1
    if c==10980:
        columnas_constantes.append(columna)

#obtenemos todas las columnas constantes y las guerdamos en un nuevo df
df_cc=df_suicidio_oms[columnas_constantes].head(n=1)
df_cc

,IndicatorCode,Indicator,ValueType,Location type,Period type,Dim1 type,Dim2 type,Dim2,Dim2ValueCode,Language,DateModified
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,Country,Year,Sex,Age Group,All age groups (total),YEARSALL,EN,2022-12-08


In [87]:
#eliminamos las columnas constantes ya que podrian ser ineficientes en terminos de memoria
df_suicidio_oms.drop(columnas_constantes,axis=1,inplace=True)
df_suicidio_oms

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,IsLatestYear,Dim1,Dim1ValueCode,Dim3 type,Dim3,Dim3ValueCode,DataSourceDimValueCode,DataSource,FactValueNumericPrefix,FactValueNumeric,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments
0,AFR,Africa,AGO,Angola,2019,True,Both sexes,BTSX,NaN,NaN,NaN,NaN,NaN,NaN,6.08,NaN,NaN,3.63,NaN,9.62,6.08 [3.63 – 9.62],NaN,NaN
1,AFR,Africa,AGO,Angola,2019,True,Male,MLE,NaN,NaN,NaN,NaN,NaN,NaN,9.95,NaN,NaN,6.02,NaN,15.62,9.95 [6.02 – 15.62],NaN,NaN
2,AFR,Africa,AGO,Angola,2019,True,Female,FMLE,NaN,NaN,NaN,NaN,NaN,NaN,2.30,NaN,NaN,1.30,NaN,3.75,2.3 [1.3 – 3.75],NaN,NaN
3,AFR,Africa,AGO,Angola,2018,False,Both sexes,BTSX,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,3.58,NaN,9.45,6 [3.58 – 9.45],NaN,NaN
4,AFR,Africa,AGO,Angola,2018,False,Male,MLE,NaN,NaN,NaN,NaN,NaN,NaN,9.80,NaN,NaN,5.92,NaN,15.33,9.8 [5.92 – 15.33],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,WPR,Western Pacific,WSM,Samoa,2001,False,Male,MLE,NaN,NaN,NaN,NaN,NaN,NaN,20.22,NaN,NaN,11.76,NaN,31.81,20.22 [11.76 – 31.81],NaN,NaN
10976,WPR,Western Pacific,WSM,Samoa,2001,False,Female,FMLE,NaN,NaN,NaN,NaN,NaN,NaN,6.59,NaN,NaN,3.67,NaN,10.55,6.59 [3.67 – 10.55],NaN,NaN
10977,WPR,Western Pacific,WSM,Samoa,2000,False,Both sexes,BTSX,NaN,NaN,NaN,NaN,NaN,NaN,13.65,NaN,NaN,7.82,NaN,21.58,13.65 [7.82 – 21.58],NaN,NaN
10978,WPR,Western Pacific,WSM,Samoa,2000,False,Male,MLE,NaN,NaN,NaN,NaN,NaN,NaN,20.26,NaN,NaN,11.70,NaN,31.86,20.26 [11.7 – 31.86],NaN,NaN


In [88]:
#de la misma manera podemos ver que las columnas 
df_suicidio_oms.columns

Index(['ParentLocationCode', 'ParentLocation', 'SpatialDimValueCode',
       'Location', 'Period', 'IsLatestYear', 'Dim1', 'Dim1ValueCode',
       'Dim3 type', 'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode',
       'DataSource', 'FactValueNumericPrefix', 'FactValueNumeric',
       'FactValueUoM', 'FactValueNumericLowPrefix', 'FactValueNumericLow',
       'FactValueNumericHighPrefix', 'FactValueNumericHigh', 'Value',
       'FactValueTranslationID', 'FactComments'],
      dtype='object')

In [91]:
df_suicidio_oms_clean=df_suicidio_oms[['ParentLocation','Location','Period','Dim1','FactValueNumeric', 'FactValueNumericLow', 'FactValueNumericHigh']]
df_suicidio_oms_clean

,ParentLocation,Location,Period,Dim1,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
0,Africa,Angola,2019,Both sexes,6.08,3.63,9.62
1,Africa,Angola,2019,Male,9.95,6.02,15.62
2,Africa,Angola,2019,Female,2.30,1.30,3.75
3,Africa,Angola,2018,Both sexes,6.00,3.58,9.45
4,Africa,Angola,2018,Male,9.80,5.92,15.33
...,...,...,...,...,...,...,...
10975,Western Pacific,Samoa,2001,Male,20.22,11.76,31.81
10976,Western Pacific,Samoa,2001,Female,6.59,3.67,10.55
10977,Western Pacific,Samoa,2000,Both sexes,13.65,7.82,21.58
10978,Western Pacific,Samoa,2000,Male,20.26,11.70,31.86


In [93]:
df_suicidio_oms_clean_por_pais = df_suicidio_oms_clean.groupby("Location")

# Obtener los datos de todos los países desde 2019 a 2002
df_suicidio_oms_clean_por_pais_2019_2002 = df_suicidio_oms_clean_por_pais.apply(lambda x: x[x["Period"].between(2019, 2000)])

# Imprimir los datos
df_suicidio_oms_clean_por_pais_2019_2002.head(10)

    

,,ParentLocation,Location,Period,Dim1,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
Location,,,,,,,,


# Preguntas: